<a href="https://colab.research.google.com/github/InTEGr8or/jupyter-fun/blob/master/nCov19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
  import pygsheets
except:
  !pip install pygsheets
  import pygsheets
try:
  import numpy
except:
  !pip install numpy
  import numpy
from dateutil import parser
try:
  import requests
except:
  !pip install requests
  import requests

try:
  from bs4 import BeautifulSoup
except:
  !pip install beautifulsoup4
  from bs4 import BeautifulSoup
  
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
pd.set_option('display.max_rows', 100)

gc = gspread.authorize(GoogleCredentials.get_application_default())
source_url = "https://docs.google.com/spreadsheets/d/1yZv9w9zRKwrGTaR-YzmAqMefw4wMlaXocejdxZaTs6w"
def get_sheets(sheet_url):
  page = requests.get(sheet_url)
  worksheets = BeautifulSoup(page.text)
  # the requests just gets the ajax scripts at this point.
  # read from '//div[@id='sheets-viewport']/div //table'
  # thats all the sheets
  pass
# get_sheets(source_url)

def sdate(worksheet):
  date_split = worksheet.title.split("_")
  time_string = ' '.join(str(x) for x in date_split) if len(date_split[1]) < 5 else date_split[0] + ' ' + date_split[1][:-4] + ':' + date_split[1][-4:]
  time_string = '2020-' + time_string
  try:
    sheet_date = parser.parse(time_string)
  except:
    print("---", worksheet.title.split("_")[1])
    sheet_date = None
  return sheet_date

def rdate(sheet_rows):
  tdate = parser.parse(sheet_rows[3][sheet_rows[0].index('Last Update')])
  return tdate.strftime("%B")[:3] + str(tdate.day)

In [106]:
#TODO: Open the public sheet in a way that doesn't require authentication, like the pd.read_csv() below.  
worksheets = gc.open_by_url(source_url).worksheets()
sheet_rows = worksheets[0].get_all_values()
prior_sheet = worksheets[0]
prior_confirmed = None
# df = pd.read_csv(source_url, dtype={0:'str', 1:'str', 2:'str'})
df = pd.DataFrame.from_records(sheet_rows[1:], columns=sheet_rows[0])
df.rename(columns={"Confirmed": rdate(sheet_rows)}, inplace=True)
for worksheet in worksheets:
  sheet_date = sdate(worksheet)
  if sheet_date.day != sdate(prior_sheet).day:
    crows = worksheet.get_all_values()
    prows = prior_sheet.get_all_values()
    prov_col = prows[0].index('Province/State')
    cnt_col = prows[0].index('Country/Region')
    conf_col = prows[0].index('Confirmed')
    dt_col = f'{sheet_date.strftime("%B")[:3]}{sheet_date.day}'
    prior_dt_col = rdate(prows)
    df[f'{prior_dt_col}%'] = '0%'
    df[dt_col] = 0
    for i, df_row in df.iterrows():
      # TODO: Find the row for the df province
      for j, row in enumerate(prows):
        if row[cnt_col] == df_row['Country/Region'] and row[prov_col] == df_row['Province/State']:
          if row[conf_col].isnumeric():
            df.at[i, dt_col] = row[conf_col]
            df.at[i, f'{prior_dt_col}%'] = f'{(round(int(df_row[prior_dt_col]) / int(row[conf_col]) * 100) - 100)}%'

  prior_sheet = worksheet
df.drop(labels='Last Update',axis=1)



,Province/State,Country/Region,Feb1,Deaths,Recovered,Feb1%,Jan31,Jan31%,Jan30,Jan30%,Jan29,Jan29%,Jan28,Jan28%,Jan27,Jan27%,Jan26,Jan26%,Jan25,Jan25%,Jan24,Jan24%,Jan23,Jan23%,Jan22
0,Hubei,Mainland China,7153,249,168,0%,7153,23%,5806,18%,4903,38%,3554,31%,2714,91%,1423,34%,1058,45%,729,33%,549,24%,444
1,Zhejiang,Mainland China,599,0,21,0%,599,11%,538,26%,428,45%,296,71%,173,35%,128,23%,104,68%,62,44%,43,59%,27
2,Guangdong,Mainland China,535,0,14,0%,535,23%,436,23%,354,28%,277,34%,207,37%,151,36%,111,42%,78,47%,53,66%,32
3,Henan,Mainland China,422,2,3,0%,422,20%,352,27%,278,35%,206,23%,168,31%,128,54%,83,159%,32,256%,9,80%,5
4,Hunan,Mainland China,389,0,8,0%,389,17%,332,20%,277,25%,221,55%,143,43%,100,45%,69,60%,43,79%,24,167%,9
5,Anhui,Mainland China,297,0,5,0%,297,25%,237,18%,200,32%,152,43%,106,51%,70,17%,60,54%,39,160%,15,67%,9
6,Jiangxi,Mainland China,286,0,9,0%,286,19%,240,48%,162,49%,109,51%,72,50%,48,33%,36,100%,18,157%,7,0%,7
7,Chongqing,Mainland China,247,1,3,0%,247,17%,211,16%,182,24%,147,11%,132,20%,110,47%,75,32%,57,111%,27,200%,9
8,Sichuan,Mainland China,207,1,3,0%,207,17%,177,25%,142,31%,108,20%,90,30%,69,57%,44,57%,28,87%,15,88%,8
9,Shandong,Mainland China,206,0,3,0%,206,12%,184,16%,158,22%,130,37%,95,27%,75,63%,46,119%,21,133%,9,50%,6
